In [1]:
import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

Python version
3.8.19 (default, Mar 20 2024, 15:27:52) 
[Clang 14.0.6 ]
Version info.
sys.version_info(major=3, minor=8, micro=19, releaselevel='final', serial=0)


# Plate2Recipe Project


## Data Collection and Preprocessing


- Data loading
- Data cleaning
- Data augmentation

In [6]:
# Load and Transform Data

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

root_dir = './data/'

train_dataset = datasets.Food101(root=root_dir, split='train', transform=train_transforms, download=True)
valid_dataset = datasets.Food101(root=root_dir, split='test', transform=valid_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

Using downloaded and verified file: ./data/food-101.tar.gz
Extracting ./data/food-101.tar.gz to ./data/


In [18]:
import pandas as pd

recipes = pd.read_csv('./data/full_dataset.csv', encoding='UTF-8')
recipes = recipes.drop(recipes.columns[0], axis=1)
recipes.head(10)

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""baking potatoes"", ""extra lean ground beef"", ..."
6,Rhubarb Coffee Cake,"[""1 1/2 c. sugar"", ""1/2 c. butter"", ""1 egg"", ""...","[""Cream sugar and butter."", ""Add egg and beat ...",www.cookbooks.com/Recipe-Details.aspx?id=210288,Gathered,"[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou..."
7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,Gathered,"[""cream-style corn"", ""whole kernel corn"", ""cra..."
8,Nolan'S Pepper Steak,"[""1 1/2 lb. round steak (1-inch thick), cut in...","[""Roll steak strips in flour."", ""Brown in skil...",www.cookbooks.com/Recipe-Details.aspx?id=375254,Gathered,"[""tomatoes"", ""water"", ""onions"", ""Worcestershir..."
9,Millionaire Pie,"[""1 large container Cool Whip"", ""1 large can c...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,Gathered,"[""pineapple"", ""condensed milk"", ""lemons"", ""pec..."


## Model Development
- CNN for Image Processing
- NLP for Recipe Generation


### NLP Model
- Assume list of ingredients is provided, we can use this to generate the recipe. 
- Match input list of ingredients with the ingredients in the dataset and generate the recipe
- The chosen recipe will be the one with the highest similarity score

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assume 'user_ingredients' list is already defined
user_ingredients = ['chicken', 'onion', 'garlic', 'salt', 'pepper', 'olive oil']

# Preprocess data
def preprocess(ingredients):
    return [' '.join(eval(ing)).lower().replace('[^a-z\s]', '') for ing in ingredients]

# Vectorize ingredients
vectorizer = TfidfVectorizer()
recipe_ingredients = preprocess(recipes['NER'])
recipe_tfidf = vectorizer.fit_transform(recipe_ingredients)

user_ingredients_string = ' '.join(user_ingredients).lower()
user_tfidf = vectorizer.transform([user_ingredients_string])

# Compute similarity
similarity_scores = cosine_similarity(user_tfidf, recipe_tfidf)

# Select the best matching recipe
best_match_index = similarity_scores.argmax()
best_recipe = recipes.iloc[best_match_index]

print("Best Matching Recipe:", best_recipe['title'])
print("Ingredients:", best_recipe['ingredients'])
print("Directions:", best_recipe['directions'])

Best Matching Recipe: Spachcocked Chicken
Ingredients: ["1 Whole Chicken", "2 tbsp Olive Oil", "2 tsp Garlic Powder", "2 tsp Onion Powder", "1 tsp Salt", "1/2 tsp Pepper"]
Directions: ["Set up smoker with your favorite wood.", "( I perfer hickory)Bring to around 350 - 400F.", "Use poultry shears or kitchen shears and cut out chickens spine.", "Turn over and press down on breast bone until flat and insert meat skewers to hold everything in place ( like the picture)", "In a bowl mix together olive oil and seasoning then brush onto prepared chicken.", "Smoke for about 2 hours.", "Dish up and enjoy."]


## Training
- Model compilation
- Model training
- Hyperparameter tuning

## Evaluation
- Model evaluation metrics
- Visualization of results

## Conclusion
- Summary of findings
- Future work